# MSP projekt 2 - Ondřej Lukášek (xlukas15)

## Úkol 1 - Věrohodnost

Začnu tím, že si naimportuji potřebné knihovny, se kterými budu v projektu pracovat.

In [2]:
import numpy as np
import pandas as pd
import scipy.optimize as opt

Následně si načtu sheet, který obsahuje data, se kterými budu pracovat.

Předpřipravím si parametry tak, abych je mohl rovnou použít dál.

In [6]:
file_path = 'Data_2024.xlsx'
sheet_name = 'Data_věrohodnost'

data = pd.read_excel(file_path, sheet_name=sheet_name)
times = data['doba práce v oboru [roky]'].dropna().to_numpy()
censored = data['censored'].to_numpy()

initial_params = (1.5, 5.0)

### Bod 1

In [ ]:
def weibull_log_likelihood(params, times, censored):
    k, lam = params
    
    # logaritmizace pro necenzurovana data
    log_f = np.log(k) - k * np.log(lam) + (k - 1) * np.log(times) - (times / lam)**k
    # logaritmizace pro cenzurovana data
    log_sf = - (times / lam)**k
    
    # spojeni logaritmu pro necenzurovana a cenzurovana data
    likelihood = (1 - censored) * log_f + censored * log_sf

    return likelihood.sum()


# MOZNA SMAZAT???
def weibull_log_likelihood_derivatives(params, times, censored):
    k, lam = params

    # predvypocitani konstant, co se budou v derivacich opakovat
    t_lam_k = (times / lam)**k
    log_t_lam = np.log(times / lam)
    
    # parcialni derivace logaritmu podle parametru k
    dL_dk = np.sum((1 - censored) * (1 / k + log_t_lam - t_lam_k * log_t_lam) + censored * (-t_lam_k * log_t_lam))
    # parcialni derivace logaritmu podle parametru lambda
    dL_dlam = np.sum((1 - censored) * (-k / lam + k * t_lam_k / lam) + censored * (k * t_lam_k / lam))
    
    return np.array([dL_dk, dL_dlam])


log_likelihood = weibull_log_likelihood(initial_params, times, censored)
derivatives = weibull_log_likelihood_derivatives(initial_params, times, censored)

print(f'Log-likelihood: {log_likelihood}')
print(f'Derivatives: {derivatives}')

Log-likelihood: -740.8976732574417
Derivatives: [64.24195393 77.00791218]


### Bod 2

In [ ]:
def neg_weibull_log_likelihood(params, times, censored):
    return -weibull_log_likelihood(params, times, censored)

result = opt.minimize(
    fun=neg_weibull_log_likelihood,
    x0=initial_params,
    args=(times, censored),
    method='L-BFGS-B',
    bounds=[(0.1, None), (0.1, None)]
)

optimal_shape, optimal_scale = result.x

print('Maximal likelihood estimation:')
print(f'Shape (k): {optimal_shape}')
print(f'Scale (lambda): {optimal_scale}')

Maximal likelihood estimation:
Shape (k): 6.172808847674017
Scale (lambda): 7.4294603426242265


## Úkol 2 - Regrese

In [8]:
# TODO